<a href="https://colab.research.google.com/github/polis-community/red-dwarf/blob/main/docs/notebooks/strip-pass-votes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet red-dwarf[all]@git+https://github.com/polis-community/red-dwarf.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.1/116.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 177.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 148.5 MB/s eta 0:00:00


In [1]:
CONVOS = {
    # Topic: What were the most significant developments in tech and politics in 2018?
    # 5 groups, 65 ptpts (56 grouped), 43 comments (open)
    "tech-politics-2018": {
        "report_id": "r2dfw8eambusb8buvecjt",
        "convo_id": "6jrufhr6dp",
    },
    # Topic: How should we operate vehicle-for-hire, e.g. Uber, Lyft and taxis in Toronto?
    # 2 groups, 47 ptpts (36 grouped), 69 comments (open)
    "rideshare-toronto": {
        "report_id": "r8xhmkwp6shm9yfermteh",
        "convo_id": "7vampckwrh",
    },
}

In [2]:
# Set up interactive widgets.

from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from random import random

checkbox = widgets.Checkbox(
    value=False,
    description='Remove pass votes',
    indent=True,
)
random_val = widgets.FloatText()
button = widgets.Button(description="Re-run Clustering")

def on_button_click(_):
    random_val.value = random()
button.on_click(on_button_click)

In [3]:
from reddwarf.data_loader import Loader
from reddwarf.utils.statements import process_statements
from reddwarf.implementations.base import run_pipeline
from reddwarf.data_presenter import generate_figure_polis

report_id = CONVOS["tech-politics-2018"]["report_id"]
print(f"Loading data from report: https://pol.is/report/{report_id}")
print()
print("Run with or without pass votes, to see the algorithmic effect of including pass votes.")

# Display the checkboxes.
display(widgets.HBox([checkbox, button]))


# To see the consequences of not having pass/neutral/zero votes
def cluster_and_plot(do_strip_pass_votes, dummy_var):
    loader = Loader(polis_id=report_id)
    
    if do_strip_pass_votes:
        loader.votes_data = [v for v in loader.votes_data if v["vote"] != 0]

    _, _, mod_out_statement_ids, meta_statement_ids = process_statements(loader.comments_data)
    result = run_pipeline(
        reducer="pca",
        clusterer="kmeans",
        votes=loader.votes_data,
        mod_out_statement_ids=mod_out_statement_ids,
        meta_statement_ids=meta_statement_ids,
    )
    
    generate_figure_polis(result)

# Render the interactive widget
widgets.interactive_output(cluster_and_plot, {'do_strip_pass_votes': checkbox, 'dummy_var': random_val})

Loading data from report: https://pol.is/report/r2dfw8eambusb8buvecjt

Run with or without pass votes, to see the algorithmic effect of including pass votes.


Output()